<style>div.container { width: 100% }</style>
<img style="float:left;  vertical-align:text-bottom;" height="65" width="172" src="../assets/holoviz-logo-unstacked.svg" />
<div style="float:right; vertical-align:text-bottom;"><h2>Tutorial 3: Interlinked Panels</h2></div>

In [ ]:
import panel as pn

pn.extension()

In the previous section we learned the very basics of working with Panel. Specifically we looked at the different types of components, how to update them and how to serve a Panel application or dashboard. However to start building actual apps with Panel we need to be able to add interactivity by linking different components together. In this section we will learn how to link widgets to outputs to start building some simple interactive applications.

In this section we will once again make use of the earthquake dataset we loaded previously and compute some statistics:

In [ ]:
import dask.dataframe as dd

df = dd.read_parquet('../data/earthquakes.parq', columns=['time', 'place', 'mag'])
df['time'] = df.time.dt.strftime('%m/%d/%Y %H:%M:%S') 
df = df.reset_index(drop=True).persist()

## Widgets and reactive components

`pn.interact` constructs widgets automatically that can then be reconfigured, but if you want more control, you'll want to instantiate widgets explicitly. A widget is an input control that allows a user to change a ``value`` using some graphical UI. A simple example is a `RangeSlider`:

In [ ]:
mag_filter = pn.widgets.RangeSlider(name='Magnitude', start=0, end=df.mag.max().compute())

mag_filter

Here the widget `value` is a [Parameter](https://param.pyviz.org) that is set to a tuple of the selected upper and lower bound. Parameters are an extended type of Python attribute that declare their type, range, etc. so that other code can interact with them in a consistent way. When we change the range using the widget the ``value`` parameter updates, and vice versa if you change the value parameter manually:

In [ ]:
mag_filter.value

Now we will declare a second widget:

In [ ]:
place_filter = pn.widgets.TextInput(placeholder='Enter a placename')

place_filter

In addition to the fully automated `pn.interact`, Panel offers a very concise, powerful approach of declaring dependencies between the parameters of one object and the arguments to a function. In practice, this middle ground provides enough control for nearly any app, without the complexity of explicit chains of callbacks.

Here we will take the two widgets we have instantiated and create little function that `depends` on the values of these widgets and filters the dataframe. Using the ``pn.depends`` decorator we can then declare that this function depends on the widget values (to be more explicit you can explicitly declare the dependency on the `place_filter.param.value` object).

In [ ]:
@pn.depends(mag_filter, place_filter)
def filter_df(mag_range, place):
    lower = df.mag>mag_range[0]
    upper = df.mag<mag_range[1]
    dffilter = lower & upper
    if place:
        dffilter &= df.place.str.contains(place)
    return df[dffilter].head()

Finally we lay out the widget and the function:

In [ ]:
filtered_view = pn.Row(
    pn.Column(mag_filter, place_filter),
    pn.panel(filter_df, width=400)
)

filtered_view

Whenever one of the widgets is changed the filter_df function will be triggered and the DataFrame pane will update with the updated data.

Let us also take a look at the repr():

In [ ]:
print(filtered_view)

The `ParamFunction` pane is what listens to changes in the parameters on the widgets and updates the displayed output.

#### Exercise

Declare two ``Spinner`` widgets with an initial value of 1, then declare a function that depends on the values of both widgets and adds them together. Finally lay out the two widgets and the function in a Panel:

<details><summary>Solution</summary><br>

```python
a = pn.widgets.Spinner(value=1, width=60)
b = pn.widgets.Spinner(value=1, width=60)

@pn.depends(a.param.value, b.param.value)
def adder(a, b):
    return a + b

pn.Row(a, '+', b, '=', adder)
```
    
</details>

## Callbacks

The `depends` API is still a very high level way of declaring interactive components. Panel also supports the more low-level approach of writing callbacks in response to changes in some parameter, e.g. the ``value`` of a widget. All parameters can be watched using the ``.param.watch`` API, which will call the provided callback with an event object containing the old and new value of the widget.

Now that it is loaded we will create a slider which we will eventually use to select the row of the dataframe that we want to display:

In [ ]:
row_slider = pn.widgets.IntSlider(value=0, start=0, end=len(df)-1)

Next we create a Pane to display the current row of the dataframe with times formatted nicely:

In [ ]:
row_pane = pn.panel(df.loc[row_slider.value].compute())

Now that we have defined both the widget and the object we want to update we can declare a callback to link the two. As we learned in the previous section assigning a new value to the ``object`` of a pane will update the display. In the callback we select the row of the dataframe and then assign it to the ``pane.object``.

In [ ]:
def df_callback(event):
    row_pane.object = df.loc[event.new].compute()

Lastly we actually have to register this callback. To do so we provide the callback and the parameter we want to trigger the event on the slider's ``.param.watch`` method:

In [ ]:
row_slider.param.watch(df_callback, 'value')

Now that everything is connected up we can put both the widget and the pane in a panel and display them:

In [ ]:
pn.Column(row_slider, row_pane, width=400)

As you can see, this process is slightly more laborious than `pn.interact` or even the `pn.depends` approach, but doing it in this way should help you see how everything fits together and can be useful to more precisely control callbacks that update particular parameters or the contents of a larger layout.

# Moving onwards

Now that we have learned to link parameters between displayed objects and build interactive components, we can start building actual apps and dashboards. Before we move on to plotting and visualization let us quickly use what we have learned by adding interactivity to [the dashboard we built in the previous exercise](./exercises/Building_a_Dashboard.ipynb).